### Please install the required Python modules/SDKs

In [13]:
! activate ai-azure-c1

import sys

sys.path.append("/opt/conda/envs/ai-azure-c1/lib/python3.8/site-packages")

Could not find conda environment: ai-azure-c1
You can list all discoverable environments with `conda info --envs`.



# Azure Custom Vision - Object Detection Demo

## Import utility functions and Python modules 

In [14]:
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

import os, time, uuid

In [15]:
def show_image_in_cell(img_url):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))
    plt.figure(figsize=(20,10))
    plt.imshow(img)
    plt.show()

In [16]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials


### Resources:
- Azure Custom Vision Endpoint
- Training Reource ID and Key
- Prediction Resource ID and Key

## Make sure you have the correct Training and Prediction Endpoints, Keys and Resource IDs separately

### Your Training Endpoint resource must be for both training and prediction for this demo

In [17]:
TRAINING_ENDPOINT = "https://nayanacustomvision.cognitiveservices.azure.com/"
training_key = "ce08598caf404621b37136119dccd921"
training_resource_id = '/subscriptions/bb272072-9c6d-4e28-b814-947814c3e6ef/resourceGroups/nayana-ai-msft-azure/providers/Microsoft.CognitiveServices/accounts/nayanacustomvision'

In [18]:
PREDICTION_ENDPOINT = 'https://nayanacustomvision-prediction.cognitiveservices.azure.com/'
prediction_key = "6b7c1fe2bea0431a8a3b4259749591e6"
prediction_resource_id = "/subscriptions/bb272072-9c6d-4e28-b814-947814c3e6ef/resourceGroups/nayana-ai-msft-azure/providers/Microsoft.CognitiveServices/accounts/nayanacustomvision-Prediction"

## Instantiate and authenticate the training client with endpoint and key 

In [19]:
training_credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(TRAINING_ENDPOINT, training_credentials)

In [20]:
trainer.api_version

'3.4-preview'

## Instantiate and authenticate the prediction client with endpoint and key

In [21]:
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(PREDICTION_ENDPOINT, prediction_credentials)

In [22]:
predictor.api_version

'3.1'

## Creating Training Project First

In [23]:
# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Your Object Detection Training project has been created. Please move on.")
project_name = uuid.uuid4()
project = trainer.create_project(project_name, domain_id=obj_detection_domain.id)

Your Object Detection Training project has been created. Please move on.


## Getting Project Details as collective information 

In [24]:
project.as_dict()

{'id': 'e520aaa3-5090-42dd-89c0-9a7ae5d5b743',
 'name': '26eb3b48-6102-45e0-8c35-1d2528aa7773',
 'description': '',
 'settings': {'domain_id': 'da2e3a8a-40a5-4171-82f4-58522f70fbc1',
  'classification_type': 'Multilabel',
  'target_export_platforms': [],
  'use_negative_set': True,
  'image_processing_settings': {'augmentation_methods': {'rotation': True,
    'scaling': True,
    'translation': True,
    'horizontal flip': True,
    'equalize': True,
    'solarize': True,
    'padtosquare': True}}},
 'created': '2022-06-20T11:09:08.553Z',
 'last_modified': '2022-06-20T11:09:08.553Z',
 'dr_mode_enabled': False,
 'status': 'Succeeded'}

In [25]:
project.status

'Succeeded'

## Adding Tags based on training requirements
- We have 2 tags in the training process 
  - Bird
  - Flower

In [26]:
bird_tag = trainer.create_tag(project.id, "Bird")

In [27]:
flower_tag = trainer.create_tag(project.id, "Flower")

# VERY IMPORTANT - PAUSE HERE
# Now, please go to the Custom Vision portal, upload and label your training images
## Please read the following instructions:

- The training images used in this demo can be found here: https://github.com/udacity/cd0461-building-computer-vision-solutions-with-azure-exercises/tree/main/resources/bird-flower
- Please download these images from the GitHub repo
- Please visit the Custom Vision Portal (https://computervision.api) and upload all of these images manually. 
- After that, you can label the tag region for each object directly at the portal. 
- This way, you don't need to use any third-party website or service to generate tag regions in the form of bounding box coordinates. If you have training images with those coordinates, you can use the optional section below to upload your images with the tagged objects via code.
- Once you have uploaded and labeled all the training images at the portal, you can come back to this notebook and start the training process. 

### =====================================================

## You can skip this section (between dotted lines) if you are uploading and labeling images manually at the portal.

## Optional: Uploading image with bounding box coordinates and tag via code



* If you have added bounding boxes and tags to every object to your images, you can upload these images with the tag regions via code.

* This is an alternative to uploading and labeling images directly at the Custom Vision portal. 

* Here is an example image from the URL: https://raw.githubusercontent.com/udacity/cd0461-building-computer-vision-solutions-with-azure-exercises/main/resources/image-01-boundingbox.png

In [ ]:
img_url = 'https://raw.githubusercontent.com/udacity/cd0461-building-computer-vision-solutions-with-azure-exercises/main/resources/image-01-boundingbox.png'
show_image_in_cell(img_url)

### Step #1: Create the bounding box coordinates for every tag on your images

* Make sure to add your folder full path and correct folder name of where all training images are located.

In [ ]:
local_image_path = ''

In [ ]:
!ls $local_image_path

In [ ]:
# "image_01.jpg" is the same image as the one shown above

flower_image_regions = {"image_01": [ 0.314344746162928, 0.405046480743692, 0.506493506493506, 0.34705621956618 ]}
bird_image_regions = {"image_01": [ 0.208677685950413, 0.265161575918548, 0.19185360094451, 0.581673306772908 ]}

### Step #2: Upload image with tag regions to the Custom Vision portal 

In [ ]:
tagged_images_with_regions = []

for file_name in flower_image_regions.keys():
    x,y,w,h = flower_image_regions[file_name]
    regions = [ Region(tag_id=flower_tag.id, left=x,top=y,width=w,height=h) ]
  
    with open(os.path.join (local_image_path, file_name + ".jpg"), mode="rb") as image_contents:
        tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))

for file_name in bird_image_regions.keys():
    x,y,w,h = bird_image_regions[file_name]
    regions = [ Region(tag_id=bird_tag.id, left=x,top=y,width=w,height=h) ]
  
    with open(os.path.join (local_image_path, file_name + ".jpg"), mode="rb") as image_contents:
        tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))

        
training_upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=tagged_images_with_regions))
if not training_upload_result.is_batch_successful:
    for image in training_upload_result.images:
        print("Image status: ", image.status)
    exit(-1)

### Step #3: Validate the image upload at the Custom Vision portal to make sure image uploading is done correctly with the coordinates

In [ ]:
training_upload_result.is_batch_successful

In [ ]:
# If the image with tag regions is uploaded successfully,
# you should see the image on the Custom Vision portal 
# with the proper tags and bounding boxes.

show_image_in_cell('https://raw.githubusercontent.com/udacity/cd0461-building-computer-vision-solutions-with-azure-exercises/main/resources/image-01-labeled-data-upload-verify.png')

### This is the end of the optional section.  
### =====================================================

## Once you have uploaded and labeled all the training images at the portal, you can start the training process with the code below.

## Start Object Detection Training
- We will be keep checking every 10 seconds the training progress

In [ ]:
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    print ("Waiting 10 seconds...")
    time.sleep(10)

## After training is complete, we will check model performance

In [ ]:
iteration.as_dict()

In [ ]:
iteration_list = trainer.get_iterations(project.id)
for iteration_item in iteration_list:
    print(iteration_item)

In [ ]:
model_perf = trainer.get_iteration_performance(project.id, iteration_list[0].id)

In [ ]:
model_perf.as_dict()

## Publishing the Model to the Project Endpoint

In [ ]:
## Setting the Iteration Name, this will be used when Model training is completed
publish_iteration_name = "udacity-2-classes-object-detection-custom"

In [ ]:
# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

## Performing Prediction
- Using the predictor object 

In [ ]:
local_image_path = '/home/workspace'

In [ ]:
!ls $local_image_path

In [ ]:
def perform_prediction(image_file_name):
    with open(os.path.join (local_image_path,  image_file_name), "rb") as image_contents:
        results = predictor.detect_image(project.id, publish_iteration_name, image_contents.read())
        # Display the results.
        for prediction in results.predictions:
            print("\t" + prediction.tag_name +
                  ": {0:.2f}%".format(prediction.probability * 100))

In [ ]:
# file_name = "ENTER LOCAL TEST IMAGE FILE NAME HERE"
# You can upload a test image and replace "flower.jpg" with the name of your own image

file_name = "flower.jpg"

In [ ]:
perform_prediction(file_name)

In [ ]:
## Checking the Image
with open(os.path.join (local_image_path, file_name), 'rb') as img_code:
    img_view_ready = Image.open(img_code)
    plt.figure()
    plt.imshow(img_view_ready)

In [ ]:
# file_name_2 = "ENTER LOCAL TEST IMAGE FILE NAME HERE"
# You can upload a test image and replace "bird.jpg" with the name of your own image

file_name_2 = "bird.jpg"

In [ ]:
perform_prediction(file_name_2)

In [ ]:
## Checking the Image
with open(os.path.join (local_image_path, file_name_2), 'rb') as img_code:
    img_view_ready = Image.open(img_code)
    plt.figure()
    plt.imshow(img_view_ready)